# __311 Service Requests in Chicago__

## Import dependencies

In [1]:
import altair as alt
import pandas as pd
import geopandas as gpd

## Helper Functions

In [29]:
def show_col_row_count(df):

    row, col = df.shape
    msg = "The dataframe currently has {} rows and {} columns."
    print(msg.format(row, col))

def drop_non_numeric(df, columns, verbose=False):

    if verbose:
        rows, _ = df.shape
        col_msg = "Removing all non-numeric rows for the following columns: {}."
        print(col_msg.format(columns))
    for col in columns:
        df = df[pd.to_numeric(df[col], errors="coerce").notnull()]
        if verbose:
            new_rows, _ = df.shape
            print("Removed {} rows from {}.".format(rows - new_rows, col))
            rows = new_rows
    return df

def drop_na(dataframe, columns, verbose=False):

    if verbose:
        rows, _ = dataframe.shape
        col_msg = "Removing all rows with NAs for the following columns: {}."
        print(col_msg.format(columns))
    df = dataframe.dropna(subset = columns)
    if verbose:
        new_rows, _ = dataframe.shape
        print("Removed {} rows.".format(rows - new_rows))
    return df

def save_columns(dataframe, columns):

    dataframe = dataframe[columns]
    return dataframe

def df_to_csv(dataframe, filename, verbose=False):

    try:
        dataframe.to_csv(filename, index=False)
        if verbose:
            print("Successfully saved to {}.".format(filename))
    except:
        if verbose:
            print("Failed to save to {}.".format(filename))

def extract_census_tract(row):
    return row.split(",")[0]

def evaluateForBivariate(row):
    # Adapted from the slack channel (Credit to Andrew McNutt)
    colorMatrix = [
        ["#64acbe", "#627f8c", "#574249"],
        ["#b0d5df", "#ad9ea5", "#985356"],
        ["#e8e8e8", "#e4acac", "#c85a5a"],
    ]
    xBoundaries = [1165.02, 1959.06]
    xIdx = 0
    if row["Number_of_Requests"] < xBoundaries[0]:
        xIdx = 0
    elif row["Number_of_Requests"] < xBoundaries[1]:
        xIdx = 1
    else:
        xIdx = 2
    yBoundaries = [8200.15, 12498.46]
    yIdx = 0
    if int(row["Household_Income"]) < yBoundaries[0]:
        yIdx = 2
    elif int(row["Household_Income"]) < yBoundaries[1]:
        yIdx = 1
    else:
        yIdx = 0

    return colorMatrix[yIdx][xIdx]

## Load the dataset - `311_Service_Requests.csv`

In [3]:
data_dir = "./data/311_Service_Requests_Chicago.csv"
dtype_dict = {
    "STREET_NUMBER": "object",
    "LEGACY_SR_NUMBER": "object",
    "PARENT_SR_NUMBER": "object",
    "SANITATION_DIVISION_DAYS": "object",
    }
df = pd.read_csv(data_dir, dtype=dtype_dict)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df["CREATED_DATE"] = pd.to_datetime(df["CREATED_DATE"], format='%m/%d/%Y %I:%M:%S %p')
df["CREATED_YEAR"] = df["CREATED_DATE"].dt.year
df["CLOSED_DATE"].fillna("01/16/2020 10:20:00 PM", inplace = True) 
df["CLOSED_DATE"]= pd.to_datetime(df["CLOSED_DATE"], format='%m/%d/%Y %I:%M:%S %p')
df["CLOSED_YEAR"] = df["CLOSED_DATE"].dt.year
df["DAYS_TAKEN"] = df["CLOSED_DATE"] - df["CREATED_DATE"]
df["DAYS_TAKEN"] = df["DAYS_TAKEN"].dt.days

In [5]:
df.sample(n = 5) 

,SR_NUMBER,SR_TYPE,SR_SHORT_CODE,OWNER_DEPARTMENT,STATUS,CREATED_DATE,LAST_MODIFIED_DATE,CLOSED_DATE,STREET_ADDRESS,CITY,...,CREATED_DAY_OF_WEEK,CREATED_MONTH,X_COORDINATE,Y_COORDINATE,LATITUDE,LONGITUDE,LOCATION,CREATED_YEAR,CLOSED_YEAR,DAYS_TAKEN
1433838,SR19-00506349,Rodent Baiting/Rat Complaint,SGA,Streets and Sanitation,Completed,2018-08-27 14:56:20,05/17/2019 01:48:45 PM,2018-08-29 13:22:08,1935 W PATTERSON AVE,CHICAGO,...,2,8,1.162645e+06,1.924240e+06,41.947779,-87.677560,"(41.94777926613673, -87.67756032960568)",2018,2018,1
1895455,SR19-01348886,Pothole in Street Complaint,PHF,CDOT - Department of Transportation,Completed,2019-04-18 10:00:40,05/17/2019 04:29:41 PM,2019-04-30 12:25:21,1125 N THROOP ST,NaN,...,5,4,1.167461e+06,1.907676e+06,41.902225,-87.660338,"(41.902224524406435, -87.6603379797049)",2019,2019,12
219700,SR18-00118210,Alley Light Out Complaint,SFA,CDOT - Department of Transportation,Completed,2018-09-07 10:20:16,06/25/2019 03:15:05 PM,2019-06-25 15:15:05,3630 N NEWLAND AVE,CHICAGO,...,6,9,1.129356e+06,1.923493e+06,41.946353,-87.799936,"(41.946353, -87.799936)",2018,2019,291
2224607,SR19-03182605,311 INFORMATION ONLY CALL,311IOC,311 City Services,Completed,2019-12-22 09:09:36,12/22/2019 09:30:48 AM,2019-12-22 09:09:37,2111 W Lexington ST,Chicago,...,1,12,1.162234e+06,1.896560e+06,41.871831,-87.679846,"(41.871831277993564, -87.67984621876099)",2019,2019,0
1030382,SR19-02827411,Weed Removal Request,SCP,Streets and Sanitation,Completed,2019-10-28 08:48:40,10/28/2019 08:48:52 AM,2019-10-28 08:48:52,7136 S SOUTH CHICAGO AVE,NaN,...,2,10,1.183075e+06,1.857833e+06,41.765102,-87.604539,"(41.7651020666732, -87.60453857428888)",2019,2019,0


In [6]:
show_col_row_count(df)

The dataframe currently has 2362297 rows and 40 columns.


In [7]:
df = drop_non_numeric(df, ["ZIP_CODE"], verbose=True)

Removing all non-numeric rows for the following columns: ['ZIP_CODE'].
Removed 17961 rows from ZIP_CODE.


In [8]:
df = drop_na(df, ["ZIP_CODE", "CREATED_DATE", "WARD", "LOCATION", "LATITUDE", "LATITUDE"], verbose=True)

Removing all rows with NAs for the following columns: ['ZIP_CODE', 'CREATED_DATE', 'WARD', 'LOCATION', 'LATITUDE', 'LATITUDE'].
Removed 0 rows.


In [9]:
columns = [
    "SR_NUMBER", "SR_TYPE", "SR_SHORT_CODE",
    "OWNER_DEPARTMENT","STATUS", "CREATED_DATE",
    "CLOSED_DATE", "ZIP_CODE", "DUPLICATE",
    "LEGACY_RECORD", "WARD", "LATITUDE",
    "LONGITUDE", "LOCATION",
    "DAYS_TAKEN", "CREATED_YEAR", "CLOSED_YEAR",
    "CREATED_HOUR",	"CREATED_DAY_OF_WEEK", "CREATED_MONTH"
]
df = save_columns(df, columns)

In [10]:
df.head(n = 5) 

,SR_NUMBER,SR_TYPE,SR_SHORT_CODE,OWNER_DEPARTMENT,STATUS,CREATED_DATE,CLOSED_DATE,ZIP_CODE,DUPLICATE,LEGACY_RECORD,WARD,LATITUDE,LONGITUDE,LOCATION,DAYS_TAKEN,CREATED_YEAR,CLOSED_YEAR,CREATED_HOUR,CREATED_DAY_OF_WEEK,CREATED_MONTH
11,SR19-02763102,Sewer Cave-In Inspection Request,AAD,DWM - Department of Water Management,Completed,2019-10-19 15:31:55,2019-11-25 13:34:43,60625,False,False,47.0,41.966001,-87.687281,"(41.96600085415316, -87.6872811861682)",36,2019,2019,15,7,10
171,SR19-02366062,311 INFORMATION ONLY CALL,311IOC,311 City Services,Completed,2019-09-02 07:55:43,2019-09-02 07:55:43,60612,False,False,28.0,41.871831,-87.679846,"(41.871831277993564, -87.67984621876099)",0,2019,2019,7,2,9
175,SR19-02366112,Aircraft Noise Complaint,AVN,Aviation,Completed,2019-09-02 08:11:53,2019-09-02 08:11:53,60666,False,False,41.0,41.994897,-87.887752,"(41.9948965489749, -87.8877516541201)",0,2019,2019,8,2,9
205,SR19-01480551,Pothole in Street Complaint,PHF,CDOT - Department of Transportation,Completed,2019-05-13 12:59:56,2019-11-02 08:01:21,60619,True,False,8.0,41.742149,-87.604865,"(41.74214930362783, -87.60486531173716)",172,2019,2019,12,2,5
586,SR19-02763151,311 INFORMATION ONLY CALL,311IOC,311 City Services,Completed,2019-10-19 15:42:07,2019-10-19 15:42:07,60612,False,False,38.0,41.953736,-87.768543,"(41.95373571422855, -87.76854307874956)",0,2019,2019,15,7,10


In [11]:
show_col_row_count(df)

The dataframe currently has 2325604 rows and 20 columns.


In [14]:
# Save the dataset

# Dataset for 2018/07/01 - 2018/12/31
df_2018 = df[(df['CREATED_DATE'] > '2018-07-01 00:00:00') & (df['CREATED_DATE'] <= '2018-12-31 23:59:59')]
filename_2018 = "./data/cleaned_311_2018.csv"
df_to_csv(df_2018, filename_2018, verbose=True)
# Dataset for 2019/07/01 - 2019/12/31
df_2019 = df[(df['CREATED_DATE'] > '2019-07-01 00:00:00') & (df['CREATED_DATE'] <= '2019-12-31 23:59:59')]
filename_2019 = "./data/cleaned_311_2019.csv"
df_to_csv(df_2019, filename_2019, verbose=True)
# Entire cleaned dataset
filename = "./data/cleaned_311.csv"
df_to_csv(df, filename, verbose=True)

Successfully saved to ./data/cleaned_311_2018.csv.
Successfully saved to ./data/cleaned_311_2019.csv.
Successfully saved to ./data/cleaned_311.csv.


## Create a dataset for Census Tract (Number of Requests and Income Level)



In [15]:
# Load Census Tract Polygon dataset
tract_dir = "./data/census_tract_boundary/tracts.geojson"
tracts = gpd.read_file(tract_dir)
tracts.crs = 'epsg:4326'
tracts.head()

,statefp10,name10,commarea_n,namelsad10,commarea,geoid10,notes,tractce10,countyfp10,geometry
0,17,8424,44,Census Tract 8424,44,17031842400,,842400,031,"MULTIPOLYGON (((-87.62405 41.73022, -87.62405 ..."
1,17,8403,59,Census Tract 8403,59,17031840300,,840300,031,"MULTIPOLYGON (((-87.68608 41.82296, -87.68607 ..."
2,17,8411,34,Census Tract 8411,34,17031841100,,841100,031,"MULTIPOLYGON (((-87.62935 41.85280, -87.62934 ..."
3,17,8412,31,Census Tract 8412,31,17031841200,,841200,031,"MULTIPOLYGON (((-87.68813 41.85569, -87.68816 ..."
4,17,8390,32,Census Tract 8390,32,17031839000,,839000,031,"MULTIPOLYGON (((-87.63312 41.87449, -87.63306 ..."


In [16]:
cleaned_311 = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.LONGITUDE, df.LATITUDE))

In [17]:
cleaned_311.sample(n=1)

,SR_NUMBER,SR_TYPE,SR_SHORT_CODE,OWNER_DEPARTMENT,STATUS,CREATED_DATE,CLOSED_DATE,ZIP_CODE,DUPLICATE,LEGACY_RECORD,...,LATITUDE,LONGITUDE,LOCATION,DAYS_TAKEN,CREATED_YEAR,CLOSED_YEAR,CREATED_HOUR,CREATED_DAY_OF_WEEK,CREATED_MONTH,geometry
2203246,SR19-03155373,311 INFORMATION ONLY CALL,311IOC,311 City Services,Completed,2019-12-17 15:46:16,2019-12-17 15:46:17,60612,False,False,...,41.871831,-87.679846,"(41.871831277993564, -87.67984621876099)",0,2019,2019,15,3,12,POINT (-87.67985 41.87183)


In [18]:
cleaned_311.crs = "epsg:4326"
cleaned_311 = cleaned_311[cleaned_311.geometry.type == 'Point']
tracts_311 = gpd.sjoin(cleaned_311, tracts, how="inner", op='intersects')

In [19]:
tracts_311.sample(n=1)

,SR_NUMBER,SR_TYPE,SR_SHORT_CODE,OWNER_DEPARTMENT,STATUS,CREATED_DATE,CLOSED_DATE,ZIP_CODE,DUPLICATE,LEGACY_RECORD,...,index_right,statefp10,name10,commarea_n,namelsad10,commarea,geoid10,notes,tractce10,countyfp10
841396,SR19-02535526,311 INFORMATION ONLY CALL,311IOC,311 City Services,Completed,2019-09-20 13:29:29,2019-09-20 13:29:30,60612,False,False,...,5,17,8382,28,Census Tract 8382,28,17031838200,,838200,031


In [20]:
tracts_count = tracts_311.groupby(["namelsad10"]).size().reset_index()
tracts_count = tracts_count.rename(columns={0: "Number_of_Requests"})

In [21]:
tracts_count.sample(n=1)

,namelsad10,Number_of_Requests
464,Census Tract 5803,906


In [22]:
# load income dataset
income_dir = "./data/income_by_census_tract/income_2018_by_census_tract.csv"
income_df = pd.read_csv(income_dir)
income_df.sample(n=1)

,GEO_ID,NAME,S1901_C01_001E,S1901_C01_001M,S1901_C02_001E,S1901_C02_001M,S1901_C03_001E,S1901_C03_001M,S1901_C04_001E,S1901_C04_001M,...,S1901_C04_015E,S1901_C04_015M,S1901_C01_016E,S1901_C01_016M,S1901_C02_016E,S1901_C02_016M,S1901_C03_016E,S1901_C03_016M,S1901_C04_016E,S1901_C04_016M
1036,1400000US17031821001,"Census Tract 8210.01, Cook County, Illinois",1631,85,1166,108,872,130,465,113,...,(X),(X),(X),(X),(X),(X),(X),(X),37.6,(X)


In [24]:
# Extract Census Tract
income_df["NAME"] = income_df["NAME"].apply(lambda x : extract_census_tract(x))

# Save Estimate!!Households!!Mean income (dollars)
columns = ["NAME", "S1901_C01_013M"]
income_df = save_columns(income_df, columns)
income_df = income_df.rename(columns={"NAME": "namelsad10", "S1901_C01_013M": "Household_Income"})

In [25]:
# Merge it with tracts_count
tracts_count = tracts_count.merge(income_df, on='namelsad10', how='left')
tracts_count = drop_non_numeric(tracts_count, ["Household_Income"], verbose=True)

Removing all non-numeric rows for the following columns: ['Household_Income'].
Removed 3 rows from Household_Income.


In [26]:
tracts_count.sample(n=1)

,namelsad10,Number_of_Requests,Household_Income
435,Census Tract 5206,1242,7182


In [27]:
# Save the merged dataset
tracts_311_count_filename = "./data/tracts_count.csv"
df_to_csv(tracts_count, tracts_311_count_filename, verbose=True)

Successfully saved to ./data/tracts_count.csv.


In [28]:
tracts_count

,namelsad10,Number_of_Requests,Household_Income
0,Census Tract 1001,2743,50767
1,Census Tract 1002,3270,10135
2,Census Tract 1003,3354,15620
3,Census Tract 1004,1589,11261
4,Census Tract 1005,3442,11618
...,...,...,...
794,Census Tract 8438,2215,7145
795,Census Tract 8439,2177,5744
796,Census Tract 901,1776,16964
797,Census Tract 902,3189,17667


In [30]:
tracts_count["color"] = tracts_count.apply(lambda x : evaluateForBivariate(x), axis = 1)

In [31]:
tracts_count.sample(n=3)

,namelsad10,Number_of_Requests,Household_Income,color
191,Census Tract 2503,2018,7397,#c85a5a
158,Census Tract 2312,3735,6628,#c85a5a
484,Census Tract 609,856,10415,#b0d5df


In [32]:
# Save the merged dataset
tracts_311_count_filename = "./data/tracts_count.csv"
df_to_csv(tracts_count, tracts_311_count_filename, verbose=True)

Successfully saved to ./data/tracts_count.csv.


In [33]:
tracts_311_pothole = tracts_311[(tracts_311['SR_TYPE'] == "Pothole in Street Complaint")]

In [34]:
tracts_311_pothole.sample(n=5)

,SR_NUMBER,SR_TYPE,SR_SHORT_CODE,OWNER_DEPARTMENT,STATUS,CREATED_DATE,CLOSED_DATE,ZIP_CODE,DUPLICATE,LEGACY_RECORD,...,index_right,statefp10,name10,commarea_n,namelsad10,commarea,geoid10,notes,tractce10,countyfp10
54666,SR19-01638946,Pothole in Street Complaint,PHF,CDOT - Department of Transportation,Completed,2019-06-05 14:17:25,2019-10-09 14:32:20,60625,False,False,...,629,17,403,4,Census Tract 403,4,17031040300,,040300,031
496243,SR19-02294678,Pothole in Street Complaint,PHF,CDOT - Department of Transportation,Completed,2019-08-24 15:10:03,2019-10-05 15:27:40,60651,True,False,...,576,17,2506,25,Census Tract 2506,25,17031250600,,250600,031
1418620,SR19-00477510,Pothole in Street Complaint,PHF,CDOT - Department of Transportation,Completed,2018-07-16 23:39:29,2018-07-17 13:20:08,60637,True,True,...,139,17,8439,42,Census Tract 8439,42,17031843900,Small area in CA 43,843900,031
1822062,SR19-01243546,Pothole in Street Complaint,PHF,CDOT - Department of Transportation,Completed,2019-03-29 19:04:09,2019-04-18 11:07:56,60655,True,False,...,273,17,7404,74,Census Tract 7404,74,17031740400,,740400,031
1818749,SR19-01238458,Pothole in Street Complaint,PHF,CDOT - Department of Transportation,Completed,2019-03-29 01:40:37,2019-04-03 15:47:45,60641,True,False,...,270,17,1502,15,Census Tract 1502,15,17031150200,,150200,031


In [35]:
# Save Pothole dataset
pothole_filename = "./data/tracts_311_pothole.csv"
df_to_csv(tracts_311_pothole, pothole_filename, verbose=True)

Successfully saved to ./data/tracts_311_pothole.csv.


In [36]:
tracts_311_pothole.shape

(110816, 31)

In [37]:
tracts_311_call = tracts_311[(tracts_311['SR_TYPE'] == "311 INFORMATION ONLY CALL")]

# Save 311 call dataset
call_filename = "./data/tracts_311_call.csv"
df_to_csv(tracts_311_call, call_filename, verbose=True)

Successfully saved to ./data/tracts_311_call.csv.


In [38]:
tracts_311_call.sample(n=5)

,SR_NUMBER,SR_TYPE,SR_SHORT_CODE,OWNER_DEPARTMENT,STATUS,CREATED_DATE,CLOSED_DATE,ZIP_CODE,DUPLICATE,LEGACY_RECORD,...,index_right,statefp10,name10,commarea_n,namelsad10,commarea,geoid10,notes,tractce10,countyfp10
234035,SR19-02765164,311 INFORMATION ONLY CALL,311IOC,311 City Services,Completed,2019-10-20 04:06:47,2019-10-20 04:06:47,60612,False,False,...,5,17,8382,28,Census Tract 8382,28,17031838200,,838200,031
707711,SR19-02576285,311 INFORMATION ONLY CALL,311IOC,311 City Services,Completed,2019-09-25 17:43:46,2019-09-25 17:43:46,60612,False,False,...,5,17,8382,28,Census Tract 8382,28,17031838200,,838200,031
1763269,SR19-01155321,311 INFORMATION ONLY CALL,311IOC,311 City Services,Completed,2019-03-13 22:37:42,2019-03-13 22:37:42,60612,False,False,...,5,17,8382,28,Census Tract 8382,28,17031838200,,838200,031
1800058,SR19-01211065,311 INFORMATION ONLY CALL,311IOC,311 City Services,Completed,2019-03-23 23:40:56,2019-03-23 23:40:56,60612,False,False,...,5,17,8382,28,Census Tract 8382,28,17031838200,,838200,031
791962,SR19-02681449,311 INFORMATION ONLY CALL,311IOC,311 City Services,Completed,2019-10-08 22:10:15,2019-10-08 22:10:16,60612,False,False,...,5,17,8382,28,Census Tract 8382,28,17031838200,,838200,031
